<a href="https://colab.research.google.com/github/LLevella/sql-metodics-matherial/blob/master/open-lecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Что такое Реляционная База Данных?**

 В таблицах хранится информация об объектах, представленных в базе данных.. . . К этим данным можно получить доступ многими способами, и при этом реорганизовывать таблицы БД не требуется.

*   Набор данных с предопределенными связями между ними
*   Данные организованны в виде набора таблиц
*   Таблицы состоят из столбцов и строк
*   В каждом столбце таблицы хранится определенный тип данных
*   В каждой ячейке – значение атрибута
*   Каждая стока таблицы - набор связанных значений, относящихся к одному объекту или сущности
*   Каждая строка в таблице может быть помечена уникальным идентификатором (первичным ключом)
*   Строки из нескольких таблиц могут быть связаны с помощью внешних ключей





**Целостность данных**

Распространненные инструменты поддержания целостности данных:

*   первичные ключи
*   внешние ключи
*   ограничения







**Соответствие требованиям ACID**

*  атомарность (Atomicity)
*  единообразие (Consistency )
*  изолированность (Isolation)
*  надежность (Durability)

**Structured Query Language (SQL)**

Основной интерфейс работы с реляционными БД

1986 год -  стандарт Национального института стандартов США (ANSI)

ANSI SQL поддерживается всеми популярными ядрами РСУБД


**Structured Query Language (SQL)**

*  добавление строк данных
*  обновление строк данных
*  удаление строк данных
*  извлечение наборов данных
*  управление всеми аспектами работы базы данных


**Транзакции**

Атомарное действие над БД, переводящего ее из одного целостного состояния в другое целостное состояние

Команды завершения транзакции:
*  COMMIT - успешно завершить транзакцию
*  ROLLBACK - откатить транзакцию


**Практика**

Задачи:

1.   Узнать топ - 5 наиболее загруженных аэропортов в период с 20.09.2016 по 27.09.2016
2.   Вывести пассажиров, входящих в топ-5 по суммарному чеку за перелёт на каждое число в период с 20.09.2016 по 27.09.2016
3.  Узнать  были ли брони, по которым не совершались перелеты?




In [0]:
with fct as ( -- Обобщенное табличное выражение
	select  -- извлечение данных
             count(flight_id) as flight_count, -- агрегатная функция - счетчик
             departure_airport as airport_code -- псевдоним результирующего столбца
	from bookings.flights -- таблица-источник данных
	where actual_departure between '2016-09-20 00:00:00' and '2016-09-27 23:59:59'
	group by departure_airport -- определяет, как строки будут группироваться
	order by flight_count desc -- упорядочивание строк
	limit 5 -- не более 5 строк 
)
select airport_code, airport_name, city, flight_count
from bookings.airports
inner join fct -- внутреннее соединение таблиц - источников
using (airport_code) -- условие соединения (аналог on )

**Агрегатные функции в PostgreSQL**

avg(): находит среднее значение. Входной параметр должен представлять один из следующих типов: smallint, int, bigint, real, double precision, numeric, interval. Для целочисленнных параметров результатом будет значение типа numeric, для параметров, которые представляют число с плавающей точкой, - значение типа double precision.

bit_and(): выполняет операцию побитового умножения (операции логического И) для чисел следующих типов: smallint, int, bigint, bit. Если параметр содержит значение NULL, то возвращается NULL.

bit_or(): выполняет операцию побитового сложения (операции логического ИЛИ) для чисел следующих типов: smallint, int, bigint, bit. Если параметр содержит значение NULL, то возвращается NULL.

bool_and(): выполняет операцию логического умножения для значений типа bool. Если входные все значения равны true, то возвращается true, иначе возвращается false.

bool_or(): выполняет операцию логического сложения для значений типа bool. Если входные хотя бы одно из значений равно true, то возвращается true, иначе возвращается false.

count(*): находит количество строк в запросе

count(expression): находит количество строк в запросе, для которых expression не содержит значение NULL.

sum(): находит сумму значений

min(): находит наименьшее значение

max(): находит наибольшее значение

string_agg(expression, delimiter): соединяет с помощью delimiter все текстовые значения из expression в одну строку.

In [0]:
select count(bookings.book_ref) > 0 as answer
	from bookings 
	left join tickets on bookings.book_ref=tickets.book_ref	-- внешнее левое соединение bookings и tickets
	left join ticket_flights on tickets.ticket_no=ticket_flights.ticket_no 
	left join boarding_passes on ticket_flights.ticket_no=boarding_passes.ticket_no
where boarding_passes.boarding_no is null


**Основные виды соединений**

INNER JOIN — соединение двух таблиц, в результат которого входят строки только из обеих таблиц

OUTER JOIN — соединение двух таблиц, в результат которого обязательно входят все строки либо одной, либо обеих таблиц 

CROSS JOIN — оператор перекрёстного соединения. Каждая строка одной таблицы соединяется с каждой строкой второй таблицы, давая тем самым в результате все возможные сочетания строк двух таблиц. 


**Основные виды соединений**

![alt text](https://lukaseder.files.wordpress.com/2016/07/venn-join1.png?w=662&h=496)

In [0]:
with flycnt as (
select 
         book_ref,
         book_date, 
         total_amount/max(total_amount) 
              over ( -- определяет, как нужно разделить строки запроса для обработки оконной функцией
                  partition by  date_trunc('day', book_date) -- указывает, что строки нужно разделить по группам или разделам, объединяя в соответствии с указанным в нём выражением
              ) as pcnt 
from bookings.bookings
where book_date between '2016-09-20 00:00:00' and '2016-09-27 23:59:59'
), 
topflyers as (
	select book_ref,
          book_date, 
          pcnt, 
         row_number() over(partition by  date_trunc('day', book_date) order by pcnt) as top
	from flycnt
) 
select * from topflyers where top <= 5


**Результат**

!![alt text](https://i.ibb.co/Ctn12LG/result.png)